## Sample demo for using ESPnet-Easy!
In this notebook, we will train an ASR model on Librispeech-100 dataset.

This notebook assumes that you have already downloaded the Librispeech-100 dataset from [OpenSLR](https://www.openslr.org/12), and placed the data in `/hdd/dataset/` directory.
Please replace the `/hdd/dataset/` directory with your own path.

### Data Preparation
This notebook follows an data preparation steps written in `asr.sh`. First, we will create a dump file to store the data id, audio path, and the transcription.

ESPnet-Easy accepts several types of datasets, including:
- Dictionary-based dataset with the following structure:
  ```python
  {
    "data_id": {
        "speech": path_to_speech_file,
        "text": transcription
    },
  }
  ```
- List of datasets with the following structure:
  ```python
  [
    {
        "speech": path_to_speech_file,
        "text": transcription
    },
  ]
  ```

If you want to use a dictionary-based dataset, each `data_id` must be unique.
ESPnet-Easy also accepts a dump file already created by `asr.sh`. But in this notebook, we will craete dump file from the beginning.

In [21]:
# Need to install espnet if you don't have it
!pwd
!pip install -U ../../
!pip install torchaudio

/hdd/doc/espnets/espnet_hackathon/egsez/asr
Processing /hdd/doc/espnets/espnet_hackathon
  Preparing metadata (setup.py) ... done
  Created wheel for espnet: filename=espnet-202304-py3-none-any.whl size=1565614 sha256=9bd2cdeea074a3f06f32603c795da4919c4dfd11852cde3cf69c00c82b8cfab7
  Stored in directory: /tmp/pip-ephem-wheel-cache-dh9sbsut/wheels/7b/a7/e0/b0c28f37b63513f226cdaef929d961a6f1feee21602ae5a781
Successfully built espnet
  Attempting uninstall: espnet
    Found existing installation: espnet 202304
    Uninstalling espnet-202304:
      Successfully uninstalled espnet-202304


In [6]:
# First, we define a function to create a dataset in dictionary format.

import os
import glob


def create_dataset(data_dir):
    dataset = {}
    for chapter in glob.glob(os.path.join(data_dir, "*/*")):
        text_file = glob.glob(os.path.join(chapter, "*.txt"))[0]

        with open(text_file, "r") as f:
            lines = f.readlines()

        ids_text = {
            line.split(" ")[0]: line.split(" ", maxsplit=1)[1].replace("\n", "")
            for line in lines
        }
        audio_files = glob.glob(os.path.join(chapter, "*.flac"))
        for audio_file in audio_files:
            audio_id = os.path.basename(audio_file)[: -len(".flac")]
            dataset[audio_id] = {"speech": audio_file, "text": ids_text[audio_id]}
    return dataset

Before you create dump files, you need to specify the data types for each inputs. In this notebook, we use `speech` as the sound data and `text` as the text data. So we define the `data_inputs` dictionary to store these information.

In [3]:
data_inputs = {
    "speech": {"file": "wav.scp", "type": "sound"},
    "text": {"file": "text", "type": "text"},
}

Note that the `file` key denotes the file name for the dump file, and the `type` key denotes the type of the inputs.
The `type` must be one of the data type listed in the [DATA_TYPES](https://github.com/espnet/espnet/blob/1409d89d1ca33417a7f57e4cfa77925a4f00cc3f/espnet2/train/dataset.py#L208).

Then we generate the dump files.

In [7]:
import espnetez as ez

# Then create the dump files
DUMP_DIR = "./dump/libri100"
LIBRI_100_DIRS = [
    ["/hdd/database/librispeech-100/LibriSpeech/train-clean-100", "train"],
    ["/hdd/database/librispeech-100/LibriSpeech/dev-clean", "dev-clean"],
    ["/hdd/database/librispeech-100/LibriSpeech/dev-other", "dev-other"],
]

for d, n in LIBRI_100_DIRS:
    dump_dir = os.path.join(DUMP_DIR, n)
    if not os.path.exists(dump_dir):
        os.makedirs(dump_dir)

    dataset = create_dataset(d)
    ez.data.create_dump_file(dump_dir, dataset, data_inputs)

For the dev files, we have `dev-clean` and `dev-other` directories.
We can join them to get one dev dataset, by using `ez.data.join_dumps` function.

In [8]:
ez.data.join_dumps(
    ["dump/libri100/dev-clean", "dump/libri100/dev-other"], "dump/libri100/dev"
)

Now you have dataset files in the `dump` directory.
It looks like this:

wav.scp
```
1255-138279-0008 /hdd/database/librispeech-100/LibriSpeech/dev-other/1255/138279/1255-138279-0008.flac
1255-138279-0022 /hdd/database/librispeech-100/LibriSpeech/dev-other/1255/138279/1255-138279-0022.flac
```

text
```
1255-138279-0008 TWO THREE
1255-138279-0022 IF I SAID SO OF COURSE I WILL
```


### Train sentencepiece model

Next, we will train a sentencepiece model. We need text file for training, so let's create a training file.

In [ ]:
# generate training texts from the training data
# you can select several datasets to train sentencepiece.
ez.preprocess.prepare_sentences(["dump/libri100/train/text"], "dump/spm")

ez.preprocess.train_sentencepiece(
    "dump/spm/spm_train.txt",
    "data/bpemodel",
    vocab_size=5000,
)

Okay, we have finished the data preparation. Now we will configure training process. We can use the configuration files already created by the ESPnet contributers.

To use the configuration file, we need to create the yaml file on your local machine. For example, I will use this [e-branchformer config](train_asr_e_branchformer_size256_mlp1024_linear1024_e12_mactrue_edrop0.0_ddrop0.0.yaml).

I changed the `batch_bins` parameter from `16000000` to `1600000`, to train on my GPU (RTX2080ti).

One thing I changed from the original ESPnet in this notebook is the way we define the token list.

The original ESPnet configuration defines the token list by giving all tokens in the yaml file, but in the ESPnet-Easy, we just give the path to the vocab file.
So the configuration file for preprocessing looks like this:

Finally we can start training the model!

In [ ]:
EXP_DIR = "exp/train_asr_branchformer_e24_amp"
STATS_DIR = "exp/stats_all"

training_config = ez.config.from_yaml(
    "asr",
    "train_asr_e_branchformer_size256_mlp1024_linear1024_e12_mactrue_edrop0.0_ddrop0.0.yaml",
)
# Need to update the configuration
preprocessor_config = ez.utils.load_yaml("preprocess.yaml")
training_config.update(preprocessor_config)

# replace token list
with open(preprocessor_config["token_list"], "r") as f:
    training_config["token_list"] = [t.replace("\n", "") for t in f.readlines()]

trainer = ez.trainer.Trainer(
    "asr",
    os.path.join(DUMP_DIR, "train"),
    os.path.join(DUMP_DIR, "dev"),
    EXP_DIR,
    data_inputs,
    STATS_DIR,
    training_config,
    ngpu=1,  # you can also update configuration here
)

trainer.train()